# Historical Data Collection - Automated

**Fully Automated Data Collection Pipeline**

This notebook collects historical OHLCV data based on configuration file settings.

**Data Sources:**
- **Yahoo Finance** (default): 20+ years, no TWS connection needed, 5-10x faster
- **Interactive Brokers TWS**: 2 years max, requires active TWS connection

**Configuration:**
All settings controlled via `config/data_collection_config.yaml`:
- Data source toggle (Yahoo vs IB)
- Universe selection (dynamic, ETF, custom)
- Collection parameters (period, workers, retries)
- No hardcoded values

**Usage:**
1. Edit `config/data_collection_config.yaml` to set preferences
2. Run all cells (no manual intervention needed)
3. Data automatically saved to `data/historical/daily/`

---

In [1]:
# ============================================================================
# SETUP - Load Configuration and Initialize Collector
# ============================================================================

import sys
from pathlib import Path
import yaml

# Add src to path
notebook_dir = Path.cwd()
if notebook_dir.name == 'src':
    project_root = notebook_dir.parent
    src_path = notebook_dir
else:
    project_root = notebook_dir
    src_path = notebook_dir / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Load configuration
config_path = src_path / 'config' / 'data_collection_config.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Determine data source from config
data_source = config['data_source']

print("="*70)
print("  AUTOMATED DATA COLLECTION - CONFIG-DRIVEN")
print("="*70)
print(f"\n  Config file: {config_path.relative_to(project_root)}")
print(f"  Data source: {data_source.upper()}")

# Initialize appropriate collector based on config
if data_source == 'yahoo':
    from data.yahoo_collector import YahooCollector
    collector = YahooCollector()
    print(f"  Collector: Yahoo Finance (no TWS needed)")
    print(f"  Period: {config['yahoo']['period']}")
    print(f"  Workers: {config['yahoo']['max_workers']}")
    needs_tws = False
elif data_source == 'ib':
    from data.collector import DataCollector
    from config.config import Config as IBConfig
    from connection.ib_connection import IBConnection
    
    ib_config = IBConfig()
    ib_conn = IBConnection(ib_config)
    needs_tws = True
    print(f"  Collector: Interactive Brokers")
    print(f"  Duration: {config['ib']['duration']}")
    print(f"  ⚠️  Requires active TWS connection on port {ib_config.ib_port}")
else:
    raise ValueError(f"Unknown data source: {data_source}. Use 'yahoo' or 'ib'")

print(f"\n  Data directory: {collector.daily_dir if data_source == 'yahoo' else project_root / 'data' / 'historical' / 'daily'}")
print(f"\n[OK] Setup complete - Ready for automated collection")

  AUTOMATED DATA COLLECTION - CONFIG-DRIVEN

  Config file: src\config\data_collection_config.yaml
  Data source: YAHOO
  Collector: Yahoo Finance (no TWS needed)
  Period: 20y
  Workers: auto

  Data directory: C:\Users\User\Desktop\Projects\Quant\data\historical\daily

[OK] Setup complete - Ready for automated collection


## 1. Automated Data Collection

Fully automated collection based on config file settings.

**What happens:**
1. Reads universe settings from config (dynamic/ETF/custom)
2. Gets ticker list automatically
3. Collects data using configured source (Yahoo or IB)
4. Skips existing data based on cache settings
5. Saves all results to Parquet

**Configuration Options** (`data_collection_config.yaml`):
- `universe.source`: "dynamic" (all universe tickers), "etf" (4 tickers), or "custom"
- `yahoo.period`: "20y" for 20 years of history
- `yahoo.update_existing`: false (skip existing), true (force re-download)
- `yahoo.max_workers`: Parallel download threads

No code changes needed - just edit config file!

## Quick Test - 3 Tickers

Test the setup with a small sample before running full collection.

This will download 3 tickers (AAPL, MSFT, SPY) to verify everything works.

In [2]:
# Quick test with 3 tickers
test_tickers = ['AAPL', 'MSFT', 'SPY']

print(f"Testing data collection with {len(test_tickers)} tickers: {test_tickers}")
print()

if data_source == 'yahoo':
    test_results = collector.collect_historical_data(
        tickers=test_tickers,
        period=config['yahoo']['period'],
        update_existing=True,  # Force fresh download for test
        max_workers=config['yahoo']['max_workers'],  # Use config value (auto or specific)
    )
    
    print(f"\n[OK] Test complete: {sum(test_results.values())}/{len(test_results)} successful")
    
    # Show sample
    if test_results.get('AAPL', False):
        df = collector.load_ticker_data('AAPL')
        if df is not None:
            print(f"\nAAPL sample: {len(df)} bars, {(df['date'].max() - df['date'].min()).days / 365.25:.1f} years")
            print(df.tail(3).to_string(index=False))
            
elif data_source == 'ib':
    if not ib_conn.is_connected:
        print("[!] Connecting to TWS...")
        ib_conn.connect(timeout=15)
    
    if 'collector' not in dir() or not isinstance(collector, DataCollector):
        collector = DataCollector(ib=ib_conn.ib)
    
    test_results = collector.collect_daily_data(
        tickers=test_tickers,
        duration=config['ib']['duration'],
        update_existing=True,
    )
    
    print(f"\n[OK] Test complete")

print("\n[INFO] If test looks good, proceed to full collection below")

Testing data collection with 3 tickers: ['AAPL', 'MSFT', 'SPY']


Yahoo Finance Historical Data Collection
Period: 20y
Tickers: 3
Workers: 8 (CPU cores: 8)
[1/3] MSFT   [OK] 5032 bars, 20.0 years
[2/3] SPY    [OK] 5032 bars, 20.0 years
[3/3] AAPL   [OK] 5032 bars, 20.0 years

Complete: 3/3 tickers downloaded
Time: 25.5s (8.5s per ticker)
Failed: 0

[OK] Test complete: 3/3 successful

AAPL sample: 5032 bars, 20.0 years
symbol                      date       open       high        low      close   volume
  AAPL 2026-02-11 00:00:00-05:00 274.700012 280.179993 274.450012 275.500000 51931300
  AAPL 2026-02-12 00:00:00-05:00 275.589996 275.720001 260.179993 261.730011 81077200
  AAPL 2026-02-13 00:00:00-05:00 262.010010 262.230011 255.449997 255.779999 56229900

[INFO] If test looks good, proceed to full collection below


In [3]:
# ============================================================================
# AUTOMATED DATA COLLECTION - Config-Driven
# ============================================================================

# Get ticker list based on config
universe_config = config['universe']
universe_source = universe_config['source']

print(f"Universe source: {universe_source}")
print()

if universe_source == 'dynamic':
    # Use dynamic universe from universe_builder
    from data.universe_builder import get_unique_tickers, get_etf_tickers
    
    tickers = get_unique_tickers()
    if universe_config['include_etfs']:
        tickers = list(set(tickers + get_etf_tickers()))
    
    print(f"Using dynamic universe: {len(tickers)} tickers")
    
elif universe_source == 'etf':
    # Use ETF list from config
    tickers = universe_config['etf_tickers']
    print(f"Using ETF list: {tickers}")
    
elif universe_source == 'custom':
    # Use custom ticker list from config
    tickers = universe_config['custom_tickers']
    print(f"Using custom list: {len(tickers)} tickers")
    
else:
    raise ValueError(f"Unknown universe source: {universe_source}")

print()

# Collect data using appropriate collector
if data_source == 'yahoo':
    # Yahoo Finance collection
    yahoo_config = config['yahoo']
    
    results = collector.collect_historical_data(
        tickers=tickers,
        period=yahoo_config['period'],
        update_existing=yahoo_config['update_existing'],
        max_workers=yahoo_config['max_workers'],
    )
    
    print(f"\n[OK] Collection complete: {sum(results.values())}/{len(results)} successful")
    
elif data_source == 'ib':
    # IB TWS collection
    if not needs_tws or not ib_conn.is_connected:
        print("[!] Connecting to TWS...")
        if not ib_conn.connect(timeout=15):
            print("[ERROR] TWS connection failed - check TWS is running")
            raise RuntimeError("TWS connection required for IB data source")
    
    # Initialize IB collector if not already done
    if 'collector' not in dir() or not isinstance(collector, DataCollector):
        collector = DataCollector(ib=ib_conn.ib)
    
    ib_config_params = config['ib']
    results = collector.collect_daily_data(
        tickers=tickers,
        duration=ib_config_params['duration'],
        update_existing=config['yahoo']['update_existing'],  # Reuse same setting
    )
    
    print(f"\n[OK] Collection complete")

else:
    raise ValueError(f"Unknown data source: {data_source}")

Universe source: dynamic

Using dynamic universe: 273 tickers


Yahoo Finance Historical Data Collection
Period: 20y
Tickers: 273
Workers: 8 (CPU cores: 8)

4 tickers already have 10+ years of data

[1/269] IESC   [OK] 5032 bars, 20.0 years
[2/269] AIN    [OK] 5032 bars, 20.0 years
[3/269] LULU   [OK] 4668 bars, 18.6 years
[4/269] CSCO   [OK] 5032 bars, 20.0 years
[5/269] MO     [OK] 5032 bars, 20.0 years
[6/269] GVA    [OK] 5032 bars, 20.0 years
[7/269] DUK    [OK] 5032 bars, 20.0 years
[8/269] AEIS   [OK] 5032 bars, 20.0 years
[9/269] ECL    [OK] 5032 bars, 20.0 years
[10/269] NOW    [OK] 3426 bars, 13.6 years
[11/269] QTWO   [OK] 2995 bars, 11.9 years


$GMS: possibly delisted; no price data found  (period=20y) (Yahoo error = "No data found, symbol may be delisted")
[GMS] No data returned from Yahoo Finance


[12/269] AMD    [OK] 5032 bars, 20.0 years
[13/269] GMS    [FAIL] No data returned
[14/269] PIPR   [OK] 5032 bars, 20.0 years
[15/269] ICFI   [OK] 4875 bars, 19.4 years
[16/269] ZTS    [OK] 3279 bars, 13.0 years
[17/269] PYPL   [OK] 2670 bars, 10.6 years
[18/269] GNTX   [OK] 5032 bars, 20.0 years
[19/269] PATK   [OK] 5032 bars, 20.0 years
[20/269] TEAM   [OK] 2560 bars, 10.2 years
[21/269] SXI    [OK] 5032 bars, 20.0 years
[22/269] CVCO   [OK] 5032 bars, 20.0 years
[23/269] ASTE   [OK] 5032 bars, 20.0 years
[24/269] CAT    [OK] 5032 bars, 20.0 years
[25/269] WTS    [OK] 5032 bars, 20.0 years
[26/269] MAR    [OK] 5032 bars, 20.0 years
[27/269] CWEN   [OK] 2704 bars, 10.8 years
[28/269] FTDR   [OK] 1865 bars, 7.4 years
[29/269] KAI    [OK] 5032 bars, 20.0 years


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: PPBI"}}}


[30/269] MU     [OK] 5032 bars, 20.0 years
[31/269] AMT    [OK] 5032 bars, 20.0 years
[32/269] MRVL   [OK] 5032 bars, 20.0 years
[33/269] LIN    [OK] 5032 bars, 20.0 years
[34/269] SYK    [OK] 5032 bars, 20.0 years
[35/269] CVS    [OK] 5032 bars, 20.0 years
[36/269] ABG    [OK] 5032 bars, 20.0 years


...

[215/269] SMAR   [FAIL] No data returned
[216/269] PG     [OK] 5032 bars, 20.0 years
[217/269] CRM    [OK] 5032 bars, 20.0 years
[218/269] DASH   [OK] 1301 bars, 5.2 years
[219/269] ATKR   [OK] 2434 bars, 9.7 years
[220/269] AIT    [OK] 5032 bars, 20.0 years
[221/269] IBM    [OK] 5032 bars, 20.0 years
[222/269] SHW    [OK] 5032 bars, 20.0 years
[223/269] UFPI   [OK] 5032 bars, 20.0 years
[224/269] HNI    [OK] 5032 bars, 20.0 years
[225/269] CVX    [OK] 5032 bars, 20.0 years
[226/269] SHOO   [OK] 5032 bars, 20.0 years
[227/269] UNH    [OK] 5032 bars, 20.0 years
[228/269] BCPC   [OK] 5032 bars, 20.0 years
[229/269] ISRG   [OK] 5032 bars, 20.0 years
[230/269] SFNC   [OK] 5032 bars, 20.0 years
[231/269] BKNG   [OK] 5032 bars, 20.0 years
[232/269] BIDU   [OK] 5032 bars, 20.0 years
[233/269] EBAY   [OK] 5032 bars, 20.0 years
[234/269] TMUS   [OK] 4737 bars, 18.8 years
[235/269] CDNS   [OK] 5032 bars, 20.0 years
[236/269] EOG    [OK] 5032 bars, 20.0 years
[237/269] OKTA   [OK] 2226 bars, 8.9 

$HES: possibly delisted; no price data found  (period=20y) (Yahoo error = "No data found, symbol may be delisted")
[HES] No data returned from Yahoo Finance


[255/269] HES    [FAIL] No data returned
[256/269] SLB    [OK] 5032 bars, 20.0 years
[257/269] WDAY   [OK] 3353 bars, 13.3 years
[258/269] GE     [OK] 5032 bars, 20.0 years
[259/269] HCC    [OK] 2222 bars, 8.8 years
[260/269] COP    [OK] 5032 bars, 20.0 years
[261/269] PANW   [OK] 3412 bars, 13.6 years
[262/269] LMT    [OK] 5032 bars, 20.0 years


$ANSS: possibly delisted; no price data found  (period=20y) (Yahoo error = "No data found, symbol may be delisted")
[ANSS] No data returned from Yahoo Finance


[263/269] ANSS   [FAIL] No data returned
[264/269] PEP    [OK] 5032 bars, 20.0 years
[265/269] SO     [OK] 5032 bars, 20.0 years
[266/269] MTX    [OK] 5032 bars, 20.0 years
[267/269] JPM    [OK] 5032 bars, 20.0 years
[268/269] VSEC   [OK] 5032 bars, 20.0 years
[269/269] IWM    [OK] 5032 bars, 20.0 years

Complete: 257/269 tickers downloaded
Time: 45.7s (0.2s per ticker)
Failed: 12
Failed tickers: GMS, PPBI, TCS, CSWI, SGEN, ATVI, WBA, ARCH, CEIX, SMAR
  ... and 2 more

[OK] Collection complete: 257/269 successful


## 2. View Data Collection Status

Check what data has been collected and verify data quality.

In [4]:
# View summary of all collected data
if 'collector' in dir():
    status = collector.data_status()
    
    if len(status) > 0:
        print(f"{'='*70}")
        print(f"  DATA COLLECTION STATUS - {data_source.upper()}")
        print(f"{'='*70}")
        print(f"\n  Total tickers: {len(status)}")
        print(f"  Total size: {status['size_kb'].sum():.1f} KB ({status['size_kb'].sum()/1024:.1f} MB)")
        print(f"  Average bars per ticker: {status['bars'].mean():.0f}")
        print(f"  Average years per ticker: {status['years'].mean():.1f}")
        print()
        
        # Show first 20 tickers
        print(status.head(20).to_string(index=False))
        if len(status) > 20:
            print(f"\n  ... and {len(status) - 20} more tickers")
    else:
        print("[--] No data collected yet - run collection cell above")
else:
    print("[--] Initialize collector first")

  DATA COLLECTION STATUS - YAHOO

  Total tickers: 294
  Total size: 47481.9 KB (46.4 MB)
  Average bars per ticker: 4124
  Average years per ticker: 16.4

symbol  bars start_date   end_date  years  size_kb
  AAON  5032 2006-02-14 2026-02-13   20.0    188.6
  AAPL  5032 2006-02-14 2026-02-13   20.0    227.0
  ABBV  3300 2013-01-02 2026-02-13   13.1    139.5
   ABG  5032 2006-02-14 2026-02-13   20.0    191.4
  ABNB  1300 2020-12-10 2026-02-13    5.2     58.2
   ABT  5032 2006-02-14 2026-02-13   20.0    198.0
   ACN  5032 2006-02-14 2026-02-13   20.0    210.2
  ADBE  5032 2006-02-14 2026-02-13   20.0    210.2
   ADI  5032 2006-02-14 2026-02-13   20.0    202.3
   ADP  5032 2006-02-14 2026-02-13   20.0    209.8
  ADSK  5032 2006-02-14 2026-02-13   20.0    207.3
  ADTN  5032 2006-02-14 2026-02-13   20.0    158.2
  AEIS  5032 2006-02-14 2026-02-13   20.0    183.7
  AFRM   501 2024-02-14 2026-02-12    2.0     30.2
   AGM  5032 2006-02-14 2026-02-13   20.0    178.1
   AIN  5032 2006-02-14 2026

## 3. Preview Sample Data

Load and preview data for a specific ticker to verify format and quality.

In [5]:
# Preview data for a sample ticker
sample_ticker = "AAPL"

if 'collector' in dir():
    df = collector.load_ticker_data(sample_ticker)
    
    if df is not None:
        print(f"{'='*70}")
        print(f"  {sample_ticker} - Historical Data Sample ({data_source.upper()})")
        print(f"{'='*70}")
        print(f"\n  Total bars: {len(df)}")
        print(f"  Date range: {df['date'].min()} to {df['date'].max()}")
        print(f"  Years: {(df['date'].max() - df['date'].min()).days / 365.25:.1f}")
        print()
        print("First 5 bars:")
        print(df.head().to_string(index=False))
        print()
        print("Last 5 bars:")
        print(df.tail().to_string(index=False))
        print()
        print("Columns:", list(df.columns))
        print("Data types:", df.dtypes.to_dict())
    else:
        print(f"[--] No data found for {sample_ticker}")
        print(f"     Run collection cell to download data")
else:
    print("[--] Initialize collector first")

  AAPL - Historical Data Sample (YAHOO)

  Total bars: 5032
  Date range: 2006-02-14 00:00:00-05:00 to 2026-02-13 00:00:00-05:00
  Years: 20.0

First 5 bars:
symbol                      date     open     high      low    close     volume
  AAPL 2006-02-14 00:00:00-05:00 2.325000 2.432143 2.321429 2.415714 1160938800
  AAPL 2006-02-15 00:00:00-05:00 2.398571 2.486429 2.383929 2.472143 1159771200
  AAPL 2006-02-16 00:00:00-05:00 2.496786 2.536071 2.481429 2.520357  948175200
  AAPL 2006-02-17 00:00:00-05:00 2.510714 2.531786 2.486071 2.510357  575999200
  AAPL 2006-02-21 00:00:00-05:00 2.521071 2.528571 2.452857 2.467143  779606800

Last 5 bars:
symbol                      date       open       high        low      close   volume
  AAPL 2026-02-09 00:00:00-05:00 277.910004 278.200012 271.700012 274.619995 44623400
  AAPL 2026-02-10 00:00:00-05:00 274.890015 275.369995 272.940002 273.679993 34376900
  AAPL 2026-02-11 00:00:00-05:00 274.700012 280.179993 274.450012 275.500000 51931300
  AA

# Quantitative Trading System - Main Workflow

This notebook is the main interface for the quantitative trading system.

## Workflow Overview
1. **Setup & Initialization** - Import libraries and configure environment
2. **Configuration** - Load settings from `.env` file
3. **Connection** - Establish connection to IB Gateway/TWS
4. **Account Information** - View account details and positions
5. **Trading Operations** - Execute strategies and manage positions

---

In [6]:
# ============================================================================
# 1. SETUP & INITIALIZATION
# ============================================================================

# Enable auto-reload for module changes (no kernel restart needed)
%load_ext autoreload
%autoreload 2

# Standard library imports
import sys
import os
from pathlib import Path
from datetime import datetime
import logging

# Data analysis libraries
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# IB API
from ib_insync import IB, Stock, util

# Configure path to import our custom modules
notebook_dir = Path.cwd()
if notebook_dir.name == 'src':
    project_root = notebook_dir.parent
    src_path = notebook_dir
else:
    project_root = notebook_dir
    src_path = notebook_dir / 'src'

# Add src to Python path if not already there
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import our custom classes
from config.config import Config
from connection.ib_connection import IBConnection

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Set pandas display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

# Set matplotlib style
plt.style.use('seaborn-v0_8-darkgrid')

# ============================================================================
# 2. LOAD CONFIGURATION
# ============================================================================

# Load configuration from .env file
config = Config()

# Create IB connection object
ib_conn = IBConnection(config)

# Display summary
print("="*70)
print("  SETUP COMPLETE")
print("="*70)
print(f"\n  Project Root: {project_root}")
print(f"  Python {sys.version.split()[0]}")
print(f"  TWS:         {config.ib_host}:{config.ib_port} (Client ID: {config.ib_client_id})")
print(f"  Mode:        {config.trading_mode.upper()}")
if config.is_paper_trading:
    print("  [PAPER TRADING] Safe for testing")
else:
    print("  [LIVE TRADING] Real money at risk!")
print("\n  Ready to connect to TWS")

2026-02-16 03:39:07 - config.config - INFO - Loaded configuration from C:\Users\User\Desktop\Projects\Quant\.env


  SETUP COMPLETE

  Project Root: c:\Users\User\Desktop\Projects\Quant
  Python 3.11.1
  TWS:         127.0.0.1:7497 (Client ID: 1)
  Mode:        PAPER
  [PAPER TRADING] Safe for testing

  Ready to connect to TWS


## 2. Connect to TWS

**Prerequisites:** Trader Workstation (TWS) must be running and logged in with API access enabled.

**TWS API Settings:** File → Global Configuration → API → Settings → Enable ActiveX and Socket Clients

**Port:** Paper Trading = 7497, Live Trading = 7496

See [docs/TWS_SETUP.md](../docs/TWS_SETUP.md) for detailed setup instructions.

In [7]:
# Connect to TWS (handles reconnect automatically)
success = ib_conn.connect(timeout=15)

if success:
    print(f"[OK] Connected (Mode: {config.trading_mode.upper()})")
else:
    print("[FAIL] Connection failed - Check TWS is running and API is enabled")
    print(f"       Expected port: {config.ib_port} (TWS Paper: 7497, Live: 7496)")
    print("       See docs/TWS_SETUP.md for configuration help")

2026-02-16 03:39:08 - connection.ib_connection - INFO - Connecting to 127.0.0.1:7497 (Client ID: 1, Mode: paper)
2026-02-16 03:39:08 - ib_insync.client - INFO - Connecting to 127.0.0.1:7497 with clientId 1...
2026-02-16 03:39:10 - ib_insync.client - INFO - Disconnecting
2026-02-16 03:39:10 - ib_insync.client - ERROR - API connection failed: ConnectionRefusedError(10061, "Connect call failed ('127.0.0.1', 7497)")
2026-02-16 03:39:10 - ib_insync.client - ERROR - Make sure API port on TWS/IBG is open
2026-02-16 03:39:11 - connection.ib_connection - WARNING - Client ID 1 failed: [Errno 10061] Connect call failed ('127.0.0.1', 7497)
2026-02-16 03:39:11 - connection.ib_connection - INFO - Connecting to 127.0.0.1:7497 (Client ID: 2, Mode: paper)
2026-02-16 03:39:11 - ib_insync.client - INFO - Connecting to 127.0.0.1:7497 with clientId 2...
2026-02-16 03:39:14 - ib_insync.client - INFO - Disconnecting
2026-02-16 03:39:14 - ib_insync.client - ERROR - API connection failed: ConnectionRefusedErro

[FAIL] Connection failed - Check TWS is running and API is enabled
       Expected port: 7497 (TWS Paper: 7497, Live: 7496)
       See docs/TWS_SETUP.md for configuration help


## 3. Test Connection

In [8]:
# Test connection
if ib_conn.is_connected:
    result = ib_conn.test_connection()
    print(f"[OK] Server: v{result['server_version']} | Accounts: {result.get('accounts', [])} | Positions: {result['positions_count']}")
else:
    print("[--] Not connected - run cell above first")

[--] Not connected - run cell above first


## 4. Account Overview

In [9]:
# ============================================================================
# ACCOUNT OVERVIEW -- Summary, Positions & Portfolio
# ============================================================================

if ib_conn.is_connected:
    # --- Account Summary ---
    values = ib_conn.get_account_values()
    metrics = [
        ('NetLiquidation', 'Net Liquidation'),
        ('TotalCashValue', 'Cash'),
        ('BuyingPower', 'Buying Power'),
        ('AvailableFunds', 'Available Funds'),
    ]
    
    print("ACCOUNT SUMMARY")
    print(f"{'Metric':<20} {'Value':>15}")
    print("-" * 37)
    for key, name in metrics:
        # Try multiple currency variants (USD, CAD, etc) and plain key
        val = None
        for suffix in ['_USD', '_CAD', '_EUR', '_GBP', '']:
            lookup_key = f"{key}{suffix}"
            if lookup_key in values:
                val = values[lookup_key]
                break
        
        if val and val != 'N/A':
            try:
                print(f"{name:<20} ${float(val):>13,.2f}")
            except (ValueError, TypeError):
                print(f"{name:<20} {val:>15}")
        else:
            print(f"{name:<20} {'N/A':>15}")
    
    # --- Positions ---
    positions = ib_conn.get_positions()
    print(f"\nPOSITIONS ({len(positions)})")
    if positions:
        print(f"{'Symbol':<10} {'Qty':>10} {'Avg Cost':>12}")
        print("-" * 34)
        for pos in positions:
            print(f"{pos.contract.symbol:<10} {pos.position:>10.2f} ${pos.avgCost:>10.2f}")
    else:
        print("  No open positions")
    
    # --- Portfolio ---
    portfolio = ib_conn.get_portfolio_items()
    print(f"\nPORTFOLIO ({len(portfolio)} items)")
    if portfolio:
        print(f"{'Symbol':<10} {'Qty':>10} {'Mkt Value':>15} {'P&L':>15}")
        print("-" * 52)
        total_val, total_pnl = 0, 0
        for item in portfolio:
            sign = '+' if item.unrealizedPNL >= 0 else '-'
            print(f"{item.contract.symbol:<10} {item.position:>10.2f} ${item.marketValue:>13,.2f}  {sign}${abs(item.unrealizedPNL):>12,.2f}")
            total_val += item.marketValue
            total_pnl += item.unrealizedPNL
        print("-" * 52)
        print(f"{'TOTAL':<10} {'':<10} ${total_val:>13,.2f}   ${total_pnl:>13,.2f}")
    else:
        print("  No portfolio items")
    
    print("\n[OK] Account overview complete")
else:
    print("[--] Not connected")

[--] Not connected


## 5. Data Infrastructure

Initialize data collector and streamer. View stock universe.


In [10]:
# Initialize data modules
from data.universe_builder import get_unique_tickers, get_etf_tickers, summary as universe_summary
from data.collector import DataCollector
from data.streamer import DataStreamer

# Create instances (requires active IB connection)
if ib_conn.is_connected:
    collector = DataCollector(ib=ib_conn.ib)
    streamer = DataStreamer(ib=ib_conn.ib)
    
    # Show universe
    print(universe_summary())
    print(f"\nData directory: {collector.data_dir}")
    print("[OK] Data modules initialized")
else:
    print("[--] Connect to IB first")

[--] Connect to IB first


### 5a. Build Dynamic Universe (Optional)

Build a filtered stock universe with multi-criteria selection:
- **Live constituent fetching** from S&P 500, Nasdaq 100, Dow 30, Russell 2000
- **Multi-criteria filters**: Volume, options availability, price range, market cap
- **Top 100 per index** (400 stocks total)
- **Smart caching**: Skips already-collected data

**Note:** Currently uses curated fallback lists. To implement live fetching, see `universe_builder.py` for integration with financial APIs, Wikipedia scraping, or IB contract search.

Run this once to build your universe, then use it for data collection below.

In [11]:
# Initialize universe builder
from data.universe_builder import UniverseBuilder

if ib_conn.is_connected:
    universe_builder = UniverseBuilder(ib=ib_conn.ib)
    print(f"Universe storage: {universe_builder.universe_dir}")
    print("[OK] Universe builder initialized")
    
    # Show current universe (if already built)
    print()
    print(universe_builder.summary())
else:
    print("[--] Connect to TWS first")

[--] Connect to TWS first


In [12]:
# Build universe with multi-criteria filtering
# This will:
# 1. Fetch constituents from each index (currently uses curated fallbacks)
# 2. Apply filters: volume, options, price, market cap
# 3. Rank and select top 100 per index
# 4. Save results to data/universe/

if ib_conn.is_connected and 'universe_builder' in dir():
    # Build universe for all 4 indices
    # WARNING: This can take 30-60 min depending on IB response times
    # Set top_n_per_index to a smaller number for testing (e.g., 10)
    
    print("\nBuilding dynamic universe...")
    print("  Filters: Volume >1M, Options available, $10-$500, Market cap >$1B")
    print("  Target:  100 stocks per index")
    print()
    
    universe_data = universe_builder.build_universe(
        indices=["SP500", "NASDAQ", "DOW", "RUSSELL"],
        top_n_per_index=100,  # Use smaller number for testing (e.g., 10)
        save=True
    )
    
    # Show results
    for idx, df in universe_data.items():
        print(f"\n{idx} - Top 10:")
        print(df[["symbol", "price", "avg_volume", "volume_score"]].head(10).to_string(index=False))
else:
    print("[--] Initialize universe_builder first")

[--] Initialize universe_builder first


### 5b. Collect Historical Data (Daily)

Downloads 2 years of daily OHLCV bars. Choose your collection scope below.

**Option A:** ETFs only (4 tickers, ~1 min) — Quick test
**Option B:** Full universe (~150+ tickers, ~25-30 min) — Production dataset from curated lists
**Option C:** Dynamic universe (400 stocks, ~60-70 min first run) — Multi-criteria filtered universe

**Smart Caching:** Re-running automatically skips tickers that are already up to date (requirement #5).
Only collects missing data for maximum efficiency during development.

In [13]:
# Collect daily OHLCV data
# This respects IB rate limits (~11 sec between requests)
# Re-running will skip tickers that are already up to date

if ib_conn.is_connected:
    # Choose collection scope:
    
    # OPTION A: ETFs only (4 tickers, ~1 min) - Quick test
    # daily_results = collector.collect_daily_data(
    #     tickers=get_etf_tickers(),
    #     duration="2 Y"
    # )
    
    # OPTION B: Full universe (~150 tickers, ~25-30 min) - Full dataset
    # daily_results = collector.collect_daily_data(
    #     tickers=None,  # None = all tickers from universe.py
    #     duration="2 Y",
    #     update_existing=True  # Skip up-to-date tickers
    # )
    
    # OPTION C: Dynamic universe (400 stocks, ~60-70 min first run)
    # Only collects missing data on subsequent runs
    if 'universe_builder' in dir():
        # Get tickers from dynamic universe
        dynamic_tickers = universe_builder.get_universe_tickers(include_etfs=True)
        
        # Filter out tickers that already have data (efficiency feature)
        missing_tickers = collector.filter_missing_tickers(dynamic_tickers)
        
        print(f"Total tickers in universe: {len(dynamic_tickers)}")
        print(f"Already collected: {len(dynamic_tickers) - len(missing_tickers)}")
        print(f"Need to collect: {len(missing_tickers)}")
        print()
        
        if len(missing_tickers) > 0:
            daily_results = collector.collect_daily_data(
                tickers=missing_tickers,
                duration="2 Y",
                update_existing=True
            )
        else:
            print("[OK] All universe data already collected!")
    else:
        print("[--] Build universe first or use OPTION A/B above")
else:
    print("[--] Connect to TWS first")

[--] Connect to TWS first


### 5c. View Stored Data

In [14]:
# View data collection status
status = collector.data_status()
if len(status) > 0:
    print(f"Stored data: {len(status)} tickers")
    print(f"Total size:  {status['size_kb'].sum():.1f} KB")
    print()
    print(status.to_string(index=False))
else:
    print("No data collected yet -- run cell above first")

# Quick preview of one ticker
sample_ticker = "SPY"
sample_data = collector.load_ticker_data(sample_ticker)
if sample_data is not None:
    print(f"\n--- {sample_ticker} (last 5 bars) ---")
    print(sample_data.tail().to_string(index=False))

Stored data: 294 tickers
Total size:  47481.9 KB

symbol  bars start_date   end_date  years  size_kb
  AAON  5032 2006-02-14 2026-02-13   20.0    188.6
  AAPL  5032 2006-02-14 2026-02-13   20.0    227.0
  ABBV  3300 2013-01-02 2026-02-13   13.1    139.5
   ABG  5032 2006-02-14 2026-02-13   20.0    191.4
  ABNB  1300 2020-12-10 2026-02-13    5.2     58.2
   ABT  5032 2006-02-14 2026-02-13   20.0    198.0
   ACN  5032 2006-02-14 2026-02-13   20.0    210.2
  ADBE  5032 2006-02-14 2026-02-13   20.0    210.2
   ADI  5032 2006-02-14 2026-02-13   20.0    202.3
   ADP  5032 2006-02-14 2026-02-13   20.0    209.8
  ADSK  5032 2006-02-14 2026-02-13   20.0    207.3
  ADTN  5032 2006-02-14 2026-02-13   20.0    158.2
  AEIS  5032 2006-02-14 2026-02-13   20.0    183.7
  AFRM   501 2024-02-14 2026-02-12    2.0     30.2
   AGM  5032 2006-02-14 2026-02-13   20.0    178.1
   AIN  5032 2006-02-14 2026-02-13   20.0    175.2
   AIT  5032 2006-02-14 2026-02-13   20.0    186.3
  ALGN  5032 2006-02-14 2026-02-

### 5d. Real-Time Quotes (Optional)

Stream live market data. Works during market hours (9:30 AM - 4:00 PM ET).

**Note:** Paper trading accounts require separate market data subscriptions to get real-time quotes.
Without subscriptions, you may see delayed data (15-min) or no snapshot data.
Historical data collection (5a) works fine regardless.

In [15]:
# Get real-time snapshots for index ETFs
if ib_conn.is_connected:
    etf_quotes = streamer.get_quotes(get_etf_tickers())
    if len(etf_quotes) > 0:
        print("INDEX ETF QUOTES")
        print(etf_quotes.to_string(index=False))
    else:
        print("No quote data returned (market may be closed)")
else:
    print("[--] Connect to IB first")

[--] Connect to IB first


### 5e. Options Chain Collection

Collect options chains for volatility analysis. Required for options overlay strategy (Phase 4).
Captures strikes, expirations, Greeks, and implied volatility.

In [16]:
# Initialize options collector
from data.options import OptionsCollector

if ib_conn.is_connected:
    options_collector = OptionsCollector(ib=ib_conn.ib)
    print(f"Options storage: {options_collector.options_dir}")
    print("[OK] Options collector initialized")
else:
    print("[--] Connect to TWS first")

[--] Connect to TWS first


In [17]:
# Collect options snapshots
# Uses batch operations: batch qualify + batch market data + ATM strike filtering

if ib_conn.is_connected and 'options_collector' in dir():
    # OPTION A: ETFs only (4 symbols, ~2-4 min) - Quick test
    options_symbols = get_etf_tickers()
    
    # OPTION B: Dynamic universe (uses built universe tickers)
    # Uncomment below after building universe and collecting historical data
    # if 'universe_builder' in dir():
    #     options_symbols = universe_builder.get_universe_tickers(include_etfs=True)
    
    print(f"Collecting options for {len(options_symbols)} symbols")
    print()
    
    options_data = options_collector.collect_options_snapshot(
        symbols=options_symbols,
        expirations_count=2,       # Nearest 2 expirations
        include_calls=True,
        include_puts=True,
        strike_range_pct=0.20,     # ATM +/- 20% (cuts strikes from ~180 to ~30-40)
    )
    
    # Show sample
    if options_data:
        for symbol, df in options_data.items():
            has_data = df["bid"].notna().sum()
            print(f"\n{symbol}: {len(df)} contracts ({has_data} with market data)")
            # Show ATM options (closest to current price, with data)
            sample = df[df["bid"].notna()].head(10)
            if len(sample) > 0:
                cols = ["strike", "right", "bid", "ask", "last", "volume"]
                cols = [c for c in cols if c in sample.columns]
                print(sample[cols].to_string(index=False))
            else:
                print("  No market data (paper account may need options data subscription)")
else:
    print("[--] Connect to TWS and initialize options_collector first")

[--] Connect to TWS and initialize options_collector first


In [18]:
# View stored options snapshots
if 'options_collector' in dir():
    snapshots = options_collector.get_stored_snapshots()
    if len(snapshots) > 0:
        print(f"Stored options snapshots: {len(snapshots)}")
        print(f"Total size: {snapshots['size_kb'].sum():.1f} KB\n")
        print(snapshots.to_string(index=False))
    else:
        print("No options snapshots yet -- run collection cell above")
else:
    print("[--] Initialize options_collector first")

[--] Initialize options_collector first


## 6. Disconnect

In [19]:
# Clean up streaming subscriptions and disconnect
if ib_conn.is_connected:
    if 'streamer' in dir() and streamer.active_subscriptions:
        streamer.unsubscribe_all()
    ib_conn.disconnect()
    print("[OK] Disconnected from TWS")
else:
    print("[--] Already disconnected")

[--] Already disconnected


---

## Data Collection Notes

**First Run**: Start with ETFs only (SPY, QQQ, DIA, IWM) to verify everything works.
Then uncomment `collector.collect_daily_data()` for the full universe (~25 min).

**Incremental Updates**: Re-running `collect_daily_data()` skips tickers already up to date.

**Storage**: Parquet files in `data/historical/daily/` -- fast reads, small footprint.
Push to Git for access from Codespaces.

## Upcoming Phases

1. **Mean Reversion Engine** -- Z-score signals on the collected data
2. **ML Filter** -- Feature engineering + classifier to filter signals
3. **Options Overlay** -- IV rank, strategy selection, spread builder
4. **Risk & Integration** -- Position sizing, portfolio monitoring, full pipeline